In [1]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter

users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
specie = "human"
GENOMEDIR = "/genomics/users/marta/genomes"


Bams are processed by Sara --> look for the scripts

Outputs are in:

`/users/genomics/saraa/projectTestis/STAR/{human/macaca/mouse}`


## Stranded?


In [ ]:
%%bash -s "$users_dir" "$GENOMEDIR" "$specie"

BAMDIR=/users/genomics/saraa/projectTestis/STAR/$3
OUTDIR=$1/$3/RSeQC

mkdir $OUTDIR

for file in $BAMDIR/*Log.final.out; do
    name=${file%%Log.*}
    name=${name##*/}

    ###################### get num of uniquely aligned reads #######################
    uniq_mapped_reads=$(sed '9q;d' ${BAMDIR}/${name}Log.final.out | awk '{print $6}')
    echo $uniq_mapped_reads
    ############################## RSEQC ###########################################
    module load Miniconda3/4.9.2

    if [ "$specie" == "human" ]; then
        infer_experiment.py -r $2/Annot_files_GTF/gencode.v38.primary_assembly.annotation_gene.bed -i ${BAMDIR}/${name}Aligned.sortedByCoord.out.bam -s $uniq_mapped_reads > $OUTDIR/${name}.out
    fi
done


In [5]:
%%bash -s "$users_dir" "$specie"

OUTDIR=$1/$2/RSeQC

# output for stringtie

if [[ -f $OUTDIR/summary_stringtie.csv ]]; then
    rm $OUTDIR/summary_stringtie.csv
fi

for file in $OUTDIR/*out; do
    sample=${file##*/}
    sample=${sample%%.out*}
    echo -e $sample"\t"$(python ~/Documents/scripts/Chris_decide_strandness_parameter.py -tab $file -tool stringtie) >> $OUTDIR/summary_stringtie.csv
done


In [5]:
%%bash -s "$users_dir" "$specie"

OUTDIR=$1/$2/RSeQC

# output for featureCounts

if [[ -f $OUTDIR/summary_featureCounts.csv ]]; then
    rm $OUTDIR/summary_featureCounts.csv
fi

for file in $OUTDIR/*out; do
    sample=${file##*/}
    sample=${sample%%.out*}
    echo -e $sample"\t"$(python ~/Documents/scripts/Chris_decide_strandness_parameter.py -tab $file -tool featureCounts) >> $OUTDIR/summary_featureCounts.csv
done


In [12]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

BAMDIR=/users/genomics/saraa/projectTestis/STAR/$2

module load stringtie/2.2.0

OUTDIR=$1/$2/stringtieReconstruction
mkdir -p $OUTDIR

GENOMEDIR=$3

REF=$GENOMEDIR/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf

while read sample strand; do
    ######################################################################################################
    #####################################STRINGTIE########################################################
    if [[ $strand == "rf" ]]; then
        stringtie $BAMDIR/${sample}Aligned.sortedByCoord.out.bam -G $REF --conservative --rf -o $OUTDIR/${sample}.gtf -C $OUTDIR/${sample}_cov.gtf
    elif [[ $strand == "fr" ]]; then
        stringtie $BAMDIR/${sample}Aligned.sortedByCoord.out.bam -G $REF --conservative --fr -o $OUTDIR/${sample}.gtf -C $OUTDIR/${sample}_cov.gtf
    else
        stringtie $BAMDIR/${sample}Aligned.sortedByCoord.out.bam -G $REF --conservative -o $OUTDIR/${sample}.gtf -C $OUTDIR/${sample}_cov.gtf
    fi
done < $1/$2/RSeQC/summary_stringtie.csv



#### Make summary

In [13]:
%%bash -s "$users_dir" "$specie"
## Total transcriptome + Presumably annotated + Presumably novel
OUT=$1/$2/stringtieReconstruction/transcriptomeReconstruction.csv

if [ -f "$OUT" ] ; then
    rm "$OUT"
fi
echo -e "Sample\tTotal_transcripts\tAnnotated_transcripts\tNovel_transcript" >> $OUT

for file in $1/$2/stringtieReconstruction/*gtf; do
    if [[ ${file} != *"cov"* ]]; then 
        name="${file%%.*}"
        name="${name##*/}"
        echo -e $name"\t"$(awk '($3 == "transcript") {print}' $file | wc -l)"\t"$(awk '($3 == "transcript") {print}' $file | grep "reference" | wc -l)"\t"$(awk '($3 == "transcript") {print}' $file | grep -v "reference" | wc -l) >> $OUT
    fi
done


From here, I **need** to know if what is considered as novel is really novel. Intersect with annotated genes. Those that do not overlap at all, are novels for sure: maintain.

`$GENOMEDIR/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf`

In [14]:
%%bash -s "$users_dir" "$specie"

OUTDIR=$1/$2/stringtieReconstruction

mkdir $OUTDIR/NOoriented
mkdir $OUTDIR/oriented

for file in $OUTDIR/*.gtf; do 
    if [[ ${file} != *"cov"* ]]; then 
        if [[ ${file} != *"gene"* ]]; then 
            name=${file##*/}
            name=${name%%.*}
            #separation of novel transcripts oriented not oriented
            awk '($7 == ".") {print}' $file > $OUTDIR/NOoriented/${name}_NOoriented.gtf
            awk '($7 != ".") {print}' $file > $OUTDIR/oriented/${name}_oriented.gtf
        fi
    fi
done


**FORCING STRANDNESS**

For the novel transcripts oriented, we force orientation when checking the overlapping with the reference.

In [15]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

OUTDIR=$1/$2/stringtieReconstruction

module load BEDTools/2.30.0-GCC-10.2.0

for file in $OUTDIR/oriented/*.gtf; do 
    name=${file##*/}
    name=${name%%.*}
    #select novel transcripts that do not overlap at all with the gencode annotation (-v)
    bedtools intersect -v -s -a $file -b $3/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf > $OUTDIR/oriented/${name}_novel.gtf
done


***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has inconsistent naming convention for record:
GL000009.2	ENSEMBL	gene	56140	58376	.	-	.	gene_id "ENSG00000278704.1"; gene_type "protein_coding"; gene_name "ENSG00000278704"; level 3;

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has inconsistent naming convention for record:
GL000009.2	ENSEMBL	gene	56140	58376	.	-	.	gene_id "ENSG00000278704.1"; gene_type "protein_coding"; gene_name "ENSG00000278704"; level 3;

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has inconsistent naming convention for record:
GL000009.2	ENSEMBL	gene	56140	58376	.	-	.	gene_id "ENSG00000278704.1"; gene_type "protein_coding"; gene_name "ENSG00000278704"; level 3;

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has i

**WITHOUT FORCING STRANDNESS**

For the novel transcripts NOT oriented, we do not force orientation when checking the overlapping with the reference.

In [16]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

OUTDIR=$1/$2/stringtieReconstruction

module load BEDTools/2.30.0-GCC-10.2.0

for file in $OUTDIR/NOoriented/*.gtf; do 
    name=${file##*/}
    name=${name%%.*}

    #select novel transcripts that do not overlap at all with the gencode annotation (-v)
    bedtools intersect -v -a $file -b $3/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf > $OUTDIR/NOoriented/${name}_novel.gtf
done

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has inconsistent naming convention for record:
GL000009.2	ENSEMBL	gene	56140	58376	.	-	.	gene_id "ENSG00000278704.1"; gene_type "protein_coding"; gene_name "ENSG00000278704"; level 3;

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has inconsistent naming convention for record:
GL000009.2	ENSEMBL	gene	56140	58376	.	-	.	gene_id "ENSG00000278704.1"; gene_type "protein_coding"; gene_name "ENSG00000278704"; level 3;

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has inconsistent naming convention for record:
GL000009.2	ENSEMBL	gene	56140	58376	.	-	.	gene_id "ENSG00000278704.1"; gene_type "protein_coding"; gene_name "ENSG00000278704"; level 3;

***** WARNING: File /genomics/users/marta/genomes/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf has i

I want to see if there are "alone exons", I don't want them, only full transcripts. Check transcript line, store geneid (STRG.XX), see if later exons contain the same. If so, store in new file, if not, skip.

In [20]:
for file in os.listdir(os.path.join(users_dir,specie,"stringtieReconstruction/NOoriented")):
    if file.endswith("novel.gtf"):
        outname=file[:-4]+"_complete.gtf"
        with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/NOoriented"),file)) as inp:
            with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/NOoriented"),outname), 'w') as out:
                lines = inp.readlines()
                t_gene_id = ""
                for line in lines:
                    tab_split_line=line.split("\t")
                    coma_split_line=line.split(";")
                    gene_id=re.findall(r'"([^"]*)"',coma_split_line[0])
                    if tab_split_line[2] == "transcript":
                        t_gene_id = gene_id[0]
                        out.write(line)
                    if tab_split_line[2] == "exon":
                        if gene_id[0] == t_gene_id:
                            out.write(line)
                        else:
                            continue


In [21]:
for file in os.listdir(os.path.join(users_dir,specie,"stringtieReconstruction/oriented")):
    if file.endswith("novel.gtf"):
        outname=file[:-4]+"_complete.gtf"
        with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/oriented"),file)) as inp:
            with open(os.path.join(os.path.join(users_dir,specie,"stringtieReconstruction/oriented"),outname), 'w') as out:
                lines = inp.readlines()
                t_gene_id = ""
                for line in lines:
                    tab_split_line=line.split("\t")
                    coma_split_line=line.split(";")
                    gene_id=re.findall(r'"([^"]*)"',coma_split_line[0])
                    if tab_split_line[2] == "transcript":
                        t_gene_id = gene_id[0]
                        out.write(line)
                    if tab_split_line[2] == "exon":
                        if gene_id[0] == t_gene_id:
                            out.write(line)
                        else:
                            continue


In [22]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

## Concatenate both oriented and unoriented
OUTDIR=$1/$2/stringtieReconstruction/intersect_novel
mkdir $OUTDIR


for sample in $(cat $1/$2/stringtieReconstruction/transcriptomeReconstruction.csv | tail -n +2 | cut -f1); do
    #concatenate both results in order to obtain a gtf file with all novel transcripts considered
    cat $1/$2/stringtieReconstruction/oriented/${sample}_oriented_novel_complete.gtf $1/$2/stringtieReconstruction/NOoriented/${sample}_NOoriented_novel_complete.gtf > $OUTDIR/${sample}_novel.gtf
done


mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/stringtieReconstruction/intersect_novel’: File exists


Now we use gffcompare so that we unify all novel transcripts into a single transcriptome

In [24]:
%%bash -s "$users_dir" "$specie"

OUTDIR=$1/$2/gffcompare
mkdir $OUTDIR

module load gffcompare/0.12.2

ls $1/$2/stringtieReconstruction/intersect_novel/*_novel.gtf > $OUTDIR/list_novel_assembled_files_stranded.txt

cd $OUTDIR
gffcompare -i list_novel_assembled_files_stranded.txt -o gffcmp.assembled.stranded


    

Loading query file #1: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/stringtieReconstruction/intersect_novel/human_brain_rna_1_r1_novel.gtf
Loading query file #2: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/stringtieReconstruction/intersect_novel/human_brain_rna_2_r1_novel.gtf
Loading query file #3: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/stringtieReconstruction/intersect_novel/human_brain_rna_3_r1_novel.gtf
Loading query file #4: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/stringtieReconstruction/intersect_novel/human_liver_rna_1_r1_novel.gtf
Loading query file #5: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/stringtieReconstruction/intersect_novel/human_liver_rna_2_r1_novel.gtf
Loading query file #6: /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeRecons

In [25]:
%%bash -s "$users_dir" "$specie"

echo -e "\n"$proj
awk '$3 == "transcript" {print}' $1/$2/gffcompare/gffcmp.assembled.stranded.combined.gtf | cut -f 7 | sort | uniq -c




   1022 +
   1131 -


In [26]:
%%bash -s "$users_dir" "$specie"

## miTranscriptome
## for stranded datasets
mitranscriptome=/genomics/users/marta/genomes/miTranscriptome/mitranscriptome.v2.gtf

module load BEDTools

gffcompare=$1/$2/gffcompare/gffcmp.assembled.stranded.combined.gtf
miTranscriptomeOUTDIR=$1/$2/gffcompare/miTranscriptome

mkdir $miTranscriptomeOUTDIR

bedtools intersect -s -v -a $gffcompare -b $mitranscriptome > $miTranscriptomeOUTDIR/gffcmp.assembled.stranded.combined.NOmiTranscriptome.gtf
bedtools intersect -s -wa -wb -a $gffcompare -b $mitranscriptome > $miTranscriptomeOUTDIR/gffcmp.assembled.stranded.combined.miTranscriptome.gtf
bedtools intersect -s -u -a $gffcompare -b $mitranscriptome > $miTranscriptomeOUTDIR/gffcmp.assembled.stranded.combined.miTranscriptome.unique.gtf


***** WARNING: File /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/gffcompare/gffcmp.assembled.stranded.combined.gtf has inconsistent naming convention for record:
GL000205.2	StringTie	transcript	56105	56316	.	-	.	transcript_id "TCONS_00000001"; gene_id "XLOC_000001"; oId "STRG.32362.1"; tss_id "TSS1"; num_samples "1";

***** WARNING: File /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/gffcompare/gffcmp.assembled.stranded.combined.gtf has inconsistent naming convention for record:
GL000205.2	StringTie	transcript	56105	56316	.	-	.	transcript_id "TCONS_00000001"; gene_id "XLOC_000001"; oId "STRG.32362.1"; tss_id "TSS1"; num_samples "1";

***** WARNING: File /users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/gffcompare/gffcmp.assembled.stranded.combined.gtf has inconsistent naming convention for record:
GL000205.2	StringTie	transcript	56105	56316	.	-	.	transcript_id "TCONS_0

In [27]:
%%bash -s "$users_dir" "$specie"

awk '$3 == "transcript" {print}' $1/$2/gffcompare/miTranscriptome/gffcmp.assembled.stranded.combined.miTranscriptome.unique.gtf | cut -f 7 | sort | uniq -c


    533 +
    563 -


Let's create a new reference file with novel reconstructed transcripts

In [2]:
%%bash -s "$users_dir" "$specie" "$GENOMEDIR"

export PATH=/genomics/users/marta/tools/gff3sort-master:$PATH

## reference
REF=$3/Annot_files_GTF/gencode.v38.primary_assembly.annotation.gtf 

OUTDIR=$1/$2/newReference_Resconstructed
mkdir $OUTDIR

## gffcompare
gffcompare=$1/$2/gffcompare/gffcmp.assembled.stranded.combined.gtf

## concat
cat $gffcompare $REF > $OUTDIR/gencode.v38.gffcompare.TestisLiverBrain.annotation.gtf

#sort the newly assembled transcriptome
gff3sort.pl --precise --chr_order natural $OUTDIR/gencode.v38.gffcompare.TestisLiverBrain.annotation.gtf > $OUTDIR/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.gtf
rm $OUTDIR/gencode.v38.gffcompare.TestisLiverBrain.annotation.gtf



mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed’: File exists


## Get simplified annotation


In [4]:
## Author: Marta Espinosa Camarena
## Date: 21st May 24

## This scripts attemps to build a simplified version of an annotation file.
## By simplified we mean that we keep only 1 transcript per gene, being the longest transcript the selected one.
## Running mode:
## python get_simplified_annotation.py $absolute_path_to_annotation $outdir
## make sure the output directory has writing permissions

import pandas as pd
import sys, os
from pathlib import Path

##----------------- importing requested data -----------------##
# annotation_GTF = sys.argv[1]
annotation_GTF = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.gtf" ## testing
# outdir = sys.argv[2]
outdir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed" ## testing
outfile = os.path.basename(annotation_GTF)
outfile = outfile[:-4]

## reading reference file and separate by transcripts and genes
reference = pd.read_csv(annotation_GTF, sep="\t", header=None, comment="#")
reference['transcript_id'] = reference[8].str.extract('transcript_id "([^"]*)"', expand=True)
reference['gene_id'] = reference[8].str.extract('gene_id "([^"]*)"', expand=True)

## subset exons and calculate length
exons = reference[reference[2] == "exon"]
exons['length'] = exons[4] - exons[3]
genes = reference[reference[2] == "gene"]

## calculate length transcripts
length_tid = exons.groupby('transcript_id', as_index=False)[['length']].sum()

## make dataframe with geneid_transcriptid_length
ref_w_length = length_tid.merge(reference, on=["transcript_id"], how="right").drop_duplicates()
# ref_w_length = ref_w_length[['transcript_id','gene_id','length']]
# ref_w_length.drop_duplicates(inplace=True)
# ref_w_length.dropna(inplace=True)

## select longest transcript per gene
## Group by 'gene_id' and get the indices of the rows with the maximum 'length'
## select datafrem rows corresponding to longest trancript_id - if different transcripts for the same gene with same length, we keep the first one
max_length_indices = ref_w_length.groupby('gene_id')['length'].idxmax()

longest_transcripts_gid = ref_w_length.loc[max_length_indices]
longest_transcripts_gid = longest_transcripts_gid[['gene_id','transcript_id','length']]
longest_transcripts_gid.drop_duplicates(inplace=True) ## no duplications

reference_longest = longest_transcripts_gid.merge(ref_w_length[ref_w_length[2] != "gene"], on=['gene_id','transcript_id','length'], how="left")


full_reference = pd.concat([reference_longest, genes])
full_reference.drop(['transcript_id','gene_id','length'], axis=1, inplace=True)
full_reference.to_csv(outdir+"/"+outfile+".1transcript.gtf", sep="\t", header=None, index=None,  quoting = 3)

/tmp/ipykernel_122898/1261493031.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exons['length'] = exons[4] - exons[3]


In [8]:
%%bash -s "$users_dir" "$specie"

export PATH=/genomics/users/marta/tools/gff3sort-master:$PATH

reference=$1/$2/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.gtf
out=$1/$2/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.gtf

gff3sort.pl --precise --chr_order natural $reference > $out


## Quantification


In [11]:
%%bash -s "$users_dir" "$specie" 

###PREPARING NEEDED DATA
# PROJECT=$1
# DIR=$2/$PROJECT
# p=$3 #single-end or paired-end dataset
p="single-end"
strand="secondstrand"
CLUSTERDIR=$1/$2/featureCounts_gffcompare
mkdir $CLUSTERDIR
AnnotGTF=$1/$2/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.gtf
BAMDIR=/users/genomics/saraa/projectTestis/STAR/$2
module load Subread/2.0.3
########################

# countReadPairs may need to be removed in case of single-end reads

if [ $p == "paired-end" ]; then
    if [ $strand == "firststrand" ]; then
        featureCounts -T 10 -p -s 2 -g transcript_id -O --countReadPairs -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "secondstrand" ]; then
        featureCounts -T 10 -p -s 1 -g transcript_id -O --countReadPairs -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "unstranded" ]; then
        featureCounts -T 10 -p -s 0 -g transcript_id -O --countReadPairs -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    fi
fi
if [ $p == "single-end" ]; then
    if [ $strand == "firststrand" ]; then
        featureCounts -T 10 -s 2 -g transcript_id -O -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "secondstrand" ]; then
        featureCounts -T 10 -s 1 -g transcript_id -O -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    elif [ $strand == "unstranded" ]; then
        featureCounts -T 10 -s 0 -g transcript_id -O -a $AnnotGTF -o ${CLUSTERDIR}/gffcompare_stranded_featureCounts.txt $BAMDIR/*Aligned.sortedByCoord.out.bam 
    fi
fi


mkdir: cannot create directory ‘/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/featureCounts_gffcompare’: File exists

        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.3

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 9 BAM files                                      ||
||                                                                            ||
||                           human_brain_rna_1_r1Aligned.sortedByCoor

## TPMs

In [ ]:
from rna_seq_normalization import Normalization as Norm

In [16]:
table_of_counts = pd.DataFrame()
### modify headers

df=pd.read_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/gffcompare_stranded_featureCounts.txt"), skiprows=1, sep="\t")
df['Geneid']=df['Geneid'].str.split('.').str[0]
filter_col = [col for col in df if col.startswith('/')]
for col in filter_col:
    new_col=col.split("Aligned")[0]
    new_col=new_col.split("/")[-1]

    df.rename(columns={col:new_col}, inplace=True)
    
cols_to_select = ["Geneid","Chr","Start","End","Strand","Length"]

print(len(df.columns))
df.rename(columns={'Geneid':'gene_id'}, inplace=True)

if table_of_counts.empty:
    table_of_counts = df
else:
    table_of_counts = table_of_counts.merge(df, on=["gene_id","Chr","Start","End","Strand","Length"])

table_of_counts.to_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/table_of_counts.csv"), index=None)

# get TPMs
length = table_of_counts['Length']
genes = table_of_counts['gene_id']
# we are only interested in the columns with counts
counts = table_of_counts
counts.drop(["Chr","Start","End","Strand","Length","gene_id"],axis=1, inplace=True)
# calculate TPMs
tpm_df = Norm.tpm(counts, length)
# add gene_names again
tpms = pd.concat([genes,tpm_df], axis=1)
tpms.to_csv(os.path.join(users_dir,specie,"featureCounts_gffcompare/table_of_counts_TPMs.csv"), index=None)

15


## Run RiboNovel with the same annotation